In [1]:
!cat mst.py

import pyomo
import pyomo.opt
import pyomo.environ as pe
import pandas
import networkx

class MSTRowGeneration:
    """A class to find Minimum Spanning Tree using a row-generation algorithm."""

    def __init__(self, nfile):
        """The input is a CSV file describing the undirected network's edges."""
        self.df = pandas.read_csv(nfile)

        self.createRelaxedModel()

    def createRelaxedModel(self):
        """Create the relaxed model, without any subtour elimination constraints."""
        df = self.df
        node_set = set( list( df.startNode ) + list(df.destNode) )

        # Create the model and sets
        m = pe.ConcreteModel()

        df.set_index(['startNode','destNode'], inplace=True)
        edge_set = df.index.unique()

        m.edge_set = pe.Set(initialize=edge_set, dimen=2)
        m.node_set = pe.Set(initialize=node_set)
    
        # Define variables
        m.Y = pe.Var(m.edge_set, domain=pe.Binary)

        # Objective
        def obj_rule(m):
     

In [2]:
!python mst.py

Adding constraint for connected component:
['E', 'A', 'B', 'D']
Y[D,E] + Y[A,D] + Y[B,E] + Y[B,D] + Y[A,E] + Y[A,B]  <=  3.0
--------------

Adding constraint for connected component:
['F', 'A', 'B', 'E', 'C']
Y[B,E] + Y[C,F] + Y[E,F] + Y[E,C] + Y[A,E] + Y[A,B]  <=  4.0
--------------

Adding constraint for connected component:
['F', 'A', 'B', 'E', 'C', 'D']
Y[D,E] + Y[A,D] + Y[B,E] + Y[C,F] + Y[E,F] + Y[E,C] + Y[B,D] + Y[A,E] + Y[A,B]  <=  5.0
--------------

Y : Size=9, Index=edge_set
    Key        : Lower : Value : Upper : Fixed : Stale : Domain
    ('A', 'B') :     0 :   1.0 :     1 : False : False : Binary
    ('A', 'D') :     0 :   0.0 :     1 : False : False : Binary
    ('A', 'E') :     0 :   0.0 :     1 : False : False : Binary
    ('B', 'D') :     0 :   0.0 :     1 : False : False : Binary
    ('B', 'E') :     0 :   1.0 :     1 : False : False : Binary
    ('C', 'F') :     0 :   1.0 :     1 : False : False : Binary
    ('D', 'E') :     0 :   1.0 :     1 : False : False : Bin